In [ ]:
from keras import backend as K
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Flatten, Dense, Dropout, GlobalMaxPooling2D, Conv2D, MaxPooling2D
from get_net import get_resnet50, get_VGG16, get_efficientnet, get_InceptionResNetV2
from image_generator import get_generator
from keras.optimizers import SGD
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
import os

filepath="checkpoint/model_{epoch:03d}-{val_loss:.5f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss',verbose=1, 
                            save_best_only=True)
tbCallBack = TensorBoard(log_dir='logs',  # log 目录
                 histogram_freq=0,  # 按照何等频率（epoch）来计算直方图，0为不计算
#                  batch_size=32,     # 用多大量的数据计算直方图
                 write_graph=True,  # 是否存储网络结构图
                 write_grads=True, # 是否可视化梯度直方图
                 write_images=True,# 是否可视化参数
                 embeddings_freq=0, 
                 embeddings_layer_names=None, 
                 embeddings_metadata=None)

lrScheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=2, cooldown=1, verbose=1)

# 資料路徑
TRAIN_DATA_DIR  = 'all_samples'
# 影像大小
IMAGE_SIZE = (224, 224, 3)

# 影像類別數
NUM_CLASSES = 5

# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 64

# 凍結網路層數
FREEZE_LAYERS = 19

# Epoch 數
NUM_EPOCHS = 120

# 模型輸出儲存的檔案
WEIGHTS_FINAL = 'model-resnet50-final.h5'
# data augmentation setting
rotation_range=20
width_shift_range=0.2
height_shift_range=0.2
shear_range=0.1
zoom_range=0.1
channel_shift_range=10
horizontal_flip=True
vertical_flip=False
fill_mode='nearest'
validation_split=0.2
                                
train_generator, validation_generator = get_generator(TRAIN_DATA_DIR,
                                                      (IMAGE_SIZE[0],IMAGE_SIZE[1]),
                                                      BATCH_SIZE,
                                                      rotation_range,
                                                      width_shift_range,
                                                      height_shift_range,
                                                      shear_range,
                                                      zoom_range,
                                                      channel_shift_range,
                                                      horizontal_flip,
                                                      vertical_flip,
                                                      fill_mode,
                                                      validation_split)

def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(5, activation='softmax'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

net_final = get_VGG16(IMAGE_SIZE,FREEZE_LAYERS,NUM_CLASSES,0.5)
#net_final = define_model()
print("no. of layers: "+str(len(net_final.layers)))
# 輸出整個網路結構
print(net_final.summary())
#net_final = load_model("")
# 訓練模型
net_final.fit_generator(train_generator,
                        steps_per_epoch = train_generator.samples // BATCH_SIZE,
                        validation_data = validation_generator,
                        validation_steps = validation_generator.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                        callbacks = [checkpoint,tbCallBack,lrScheduler])

# 儲存訓練好的模型
net_final.save(WEIGHTS_FINAL)

Using TensorFlow backend.
c:\users\jk\anaconda3\envs\tf2\lib\site-packages\keras\callbacks\tensorboard_v2.py:97: UserWarning: The TensorBoard callback does not support gradients display when using TensorFlow 2.0. The `write_grads` argument is ignored.
  warnings.warn('The TensorBoard callback does not support '


Found 4000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.
Class #0 = drawings
Class #1 = hentai
Class #2 = neutral
Class #3 = porn
Class #4 = sexy
no. of layers: 23
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
